good movies generator program
supported genres: [Comedy,Fantasy,Crime,Film-Noir,Mystery,Documentary,Drama,Musical,Romance,Biography,History,Animation,Action,Adventure,Family,War,Music,
Horror,Western,Mystery,Sport,Sci-Fi,Thriller,Adult,Musical]

In [75]:
import numpy as np
import pandas as pd 

In [47]:
ratings = pd.read_csv('title.ratings.tsv.gz',delimiter='\t')
basics = pd.read_csv('title.basics.tsv.gz',delimiter='\t')
basics = basics.loc[basics['titleType']!='tvEpisode']

C:\Users\thoma\AppData\Local\Temp\ipykernel_16844\4239457989.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics = pd.read_csv('title.basics.tsv.gz',delimiter='\t')


giving new ratings to movies by modifying ratings dataframe

In [49]:
ratings['logNumVotes']=np.log(ratings['numVotes'])
ratings['logNumVotes']/=ratings['logNumVotes'].max()
ratings['averageRating']/=ratings['averageRating'].max()
ratings['newrating']=ratings['averageRating']*ratings['logNumVotes']

parameters: lenght (short,movie), genres (takes in a {} of genres), top_rated (25,50,75 percentiles, startdate

In [61]:
def filter1(lenght='movie',genres='All',date=0):
    
    #cleaning the basics dataframe out of the movies that have no 'startDate' registered
    date_presence=basics['startYear']!=r'\N'
    Basics=basics[date_presence]
    
    #setting the startdate as the minimum of all by default
    Basics['startYear'] = Basics['startYear'].astype(int)   
    if date==0:
        date=Basics['startYear'].min()
    
    #filtering now to reduce runtime
    filtered_movies=Basics.loc[(Basics['startYear']>=date) & (Basics['titleType']==lenght)]
    if genres=='All':
        return filtered_movies
    else:
        if len(genres)>3:
            return 'Error, maximum allowed: 3 genres'
        else:
            #select the rows containing the genres specified
            genre_filter = filtered_movies['genres'].apply(lambda x: all(genre in x.split(',') for genre in genres))
            filtered_movies=filtered_movies[genre_filter]
            assert filtered_movies.shape[0]!=0 
            return filtered_movies

merging the two dataframes into a unique one 

In [63]:
def merge_dataframes(filtered_movies,ratings):
    rated_movies=filtered_movies.merge(ratings, how='inner',on='tconst')
    return rated_movies

isolate the movies that have a rating in the right-hand part of the distribution, the quantile is chosen by the user

In [66]:
def quality_filter(top_percentile, rated_movies):
    assert top_percentile<=100
    quant=1-(top_percentile/100)
    quality_filter=rated_movies['newrating'].quantile(quant)
    final_movies=rated_movies[rated_movies['newrating']>=quality_filter]
    return final_movies

final function that will call filter1, merge_dataframes and quality_filter and produce a sample given by the user
all the paramters have to match

In [67]:
def generate_movies(Lenght='movie',Genres='All',Date=0,Top=1,n=10):
    filtered_movies=filter1(Lenght,Genres,Date)
    rated_movies=merge_dataframes(filtered_movies,ratings)
    final_movies=quality_filter(Top,rated_movies)
    suggestions=final_movies.sample(n)
    relevant_info=['primaryTitle','originalTitle','startYear','runtimeMinutes','genres','newrating']
    return suggestions[relevant_info]

In [78]:
generate_movies('movie',['Fantasy','Adventure'],Date=1990,Top=5,n=10)

,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,newrating
298,Alice in Wonderland,Alice in Wonderland,2010,108,"Adventure,Family,Fantasy",0.559060
202,Pirates of the Caribbean: Dead Man's Chest,Pirates of the Caribbean: Dead Man's Chest,2006,151,"Action,Adventure,Fantasy",0.674031
728,Fantastic Beasts and Where to Find Them,Fantastic Beasts and Where to Find Them,2016,132,"Adventure,Family,Fantasy",0.635758
585,Thor: The Dark World,Thor: The Dark World,2013,112,"Action,Adventure,Fantasy",0.616734
662,Star Wars: Episode VIII - The Last Jedi,Star Wars: Episode VIII - The Last Jedi,2017,152,"Action,Adventure,Fantasy",0.622344
503,Avatar: The Way of Water,Avatar: The Way of Water,2022,192,"Action,Adventure,Fantasy",0.661374
178,"The Chronicles of Narnia: The Lion, the Witch ...","The Chronicles of Narnia: The Lion, the Witch ...",2005,143,"Adventure,Family,Fantasy",0.601374
356,The Hobbit: The Desolation of Smaug,The Hobbit: The Desolation of Smaug,2013,161,"Adventure,Drama,Fantasy",0.706016
238,Prince of Persia: The Sands of Time,Prince of Persia: The Sands of Time,2010,116,"Action,Adventure,Fantasy",0.560017
154,Harry Potter and the Prisoner of Azkaban,Harry Potter and the Prisoner of Azkaban,2004,142,"Adventure,Family,Fantasy",0.714126
